# Below is the implementation of Text Rank Algorithm

In [ ]:
!pip3 install rouge

In [ ]:
import pandas as pd
import numpy as np
import networkx as nx
import nltk
import re
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import WordNetLemmatizer
import gensim.downloader as api


In [ ]:
# Download necessary NLTK resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Load GloVe model
glove_model = api.load("glove-wiki-gigaword-100")

In [ ]:
# Define the preprocess function
def clean_conversation(conversation):
    # Remove instruction tags completely
    conversation = re.sub(r'<s>\[INST\].*?\[\/INST\]</s>', '', conversation)

    # Remove additional tags if they exist
    conversation = re.sub(r'<s>.*?</s>', '', conversation)

    # Remove speaker labels including the ones with #
    conversation = re.sub(r'(Agent \d+:|Customer:|#[\w\s]*#:)', '', conversation)

    return conversation.strip()

def preprocess_text(text):
    # Remove instructional and speaker tags
    text = re.sub(r'<s>\[INST\].*?\[/INST\]</s>|Agent \d+:|Customer:|#[\w\s]*#:', '', text)


    # Convert to lowercase
    text = text.lower()

    # Remove punctuation and special characters
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)

    # Tokenize
    words = word_tokenize(text)

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    filtered_words = [word for word in words if word not in stop_words]

    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    lemmatized_words = [lemmatizer.lemmatize(word) for word in filtered_words]

    return ' '.join(lemmatized_words)

# Define the TextRank summary function
def get_word_embeddings(word):
    try:
        return glove_model[word]
    except KeyError:
        return np.zeros(100)

def textrank_summary(text, summary_percentage):
    sentences = sent_tokenize(text)
    clean_sentences = [word_tokenize(sentence.lower()) for sentence in sentences]

    # Generate sentence vectors using GloVe
    sentence_vectors = []
    for sentence in clean_sentences:
        if sentence:
            v = sum([get_word_embeddings(word) for word in sentence if word in glove_model]) / (len(sentence) + 0.001)
        else:
            v = np.zeros(100,)
        sentence_vectors.append(v)

    # Create similarity matrix
    sim_mat = np.zeros([len(sentences), len(sentences)])
    for i in range(len(sentences)):
        for j in range(len(sentences)):
            if i != j:
                sim_mat[i][j] = cosine_similarity([sentence_vectors[i]], [sentence_vectors[j]])[0,0]

    # Apply TextRank
    nx_graph = nx.from_numpy_array(sim_mat)
    scores = nx.pagerank(nx_graph)
    ranked_sentences = sorted(((scores[i], s) for i, s in enumerate(sentences)), reverse=True)
    top_n_sentences = int(len(ranked_sentences) * summary_percentage / 100)
    top_n_sentences = max(1, top_n_sentences)

    # Select top sentences and order them by appearance
    summary_sentences = [ranked_sentences[i][1] for i in range(top_n_sentences)]
    ordered_summary_sentences = sorted(summary_sentences, key=lambda s: sentences.index(s))

    # Create summary
    summary = ' '.join(ordered_summary_sentences)
    clean_summary = clean_conversation(summary)
    return clean_summary



In [ ]:
# Assuming the file path to your data file
# file_path = 'transcript.parquet'
file_path = 'validation.csv'

# Read the Parquet file
df = pd.read_csv(file_path)

# Apply the preprocessing function to the desired column
df['preprocessed_text'] = df['dialogue'].apply(preprocess_text)

# Apply the summarization function to the 'preprocessed_text' column for 20% and 30% summaries
# df['summary_20'] = df['dialogue'].apply(lambda x: textrank_summary(x, 20))
df['summary_30'] = df['dialogue'].apply(lambda x: textrank_summary(x, 30))

# Display the first few rows of the DataFrame to confirm
df.head()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


[==================================================] 100.0% 128.1/128.1MB downloaded


,id,dialogue,summary,topic,preprocessed_text,summary_30
0,dev_0,"#Person1#: Hello, how are you doing today?\n#P...",#Person2# has trouble breathing. The doctor as...,see a doctor,hello today trouble breathing lately type cold...,"Have you had any type of cold lately? No, I d..."
1,dev_1,#Person1#: Hey Jimmy. Let's go workout later t...,#Person1# invites Jimmy to go workout and pers...,do exercise,hey jimmy let go workout later today sure time...,Let's go workout later today. Sure. How abou...
2,dev_2,#Person1#: I need to stop eating such unhealth...,#Person1# plans to stop eating unhealthy foods...,healthy foods,need stop eating unhealthy food know mean ive ...,Those are the only things you eat? That's bas...
3,dev_3,#Person1#: Do you believe in UFOs?\n#Person2#:...,#Person2# believes in UFOs and can see them in...,UFOs and aliens,believe ufo course never saw stupid called ufo...,"Do you believe in UFOs? Of course, they are o..."
4,dev_4,#Person1#: Did you go to school today?\n#Perso...,#Person1# didn't go to school today. #Person2#...,go to school,go school today course didnt want didnt thats ...,"Did you go to school today? I didn't want to,..."


In [ ]:
print(df['summary_30'].iloc[0])

Have you had any type of cold lately?  No, I don ' t have any allergies that I know of.  Does this happen all the time or mostly when you are active?


In [ ]:
print(df['dialogue'].iloc[0])

#Person1#: Hello, how are you doing today?
#Person2#: I ' Ve been having trouble breathing lately.
#Person1#: Have you had any type of cold lately?
#Person2#: No, I haven ' t had a cold. I just have a heavy feeling in my chest when I try to breathe.
#Person1#: Do you have any allergies that you know of?
#Person2#: No, I don ' t have any allergies that I know of.
#Person1#: Does this happen all the time or mostly when you are active?
#Person2#: It happens a lot when I work out.
#Person1#: I am going to send you to a pulmonary specialist who can run tests on you for asthma.
#Person2#: Thank you for your help, doctor.


In [ ]:
print(df['preprocessed_text'].iloc[0])

hello today trouble breathing lately type cold lately cold heavy feeling chest try breathe allergy know allergy know happen time mostly active happens lot work going send pulmonary specialist run test asthma thank help doctor


In [ ]:
from rouge import Rouge

# Function to calculate ROUGE scores
def calculate_rouge_scores(summary, reference):
    rouge = Rouge()
    scores = rouge.get_scores(summary, reference)
    return scores[0]

# Calculate ROUGE scores for each row
# df['rouge_scores_20'] = df.apply(lambda row: calculate_rouge_scores(row['summary_20'], row['summary']), axis=1)
df['rouge_scores_30'] = df.apply(lambda row: calculate_rouge_scores(row['summary_30'], row['summary']), axis=1)

# Function to extract average scores from ROUGE scores
def extract_average_rouge_scores(rouge_scores):
    average_scores = {
        'rouge-1': {'f': 0, 'p': 0, 'r': 0},
        'rouge-2': {'f': 0, 'p': 0, 'r': 0},
        'rouge-l': {'f': 0, 'p': 0, 'r': 0}
    }
    for score in rouge_scores:
        for key in average_scores:
            average_scores[key]['f'] += score[key]['f']
            average_scores[key]['p'] += score[key]['p']
            average_scores[key]['r'] += score[key]['r']
    n = len(rouge_scores)
    for key in average_scores:
        average_scores[key]['f'] /= n
        average_scores[key]['p'] /= n
        average_scores[key]['r'] /= n
    return average_scores

# Calculate average ROUGE scores
# average_rouge_scores_20 = extract_average_rouge_scores(df['rouge_scores_20'].tolist())
average_rouge_scores_30 = extract_average_rouge_scores(df['rouge_scores_30'].tolist())

# Print average scores
# print("Average ROUGE scores for 20% summary:", average_rouge_scores_20)
print("Average ROUGE scores for 30% summary:", average_rouge_scores_30)


Average ROUGE scores for 30% summary: {'rouge-1': {'f': 0.1830719995415026, 'p': 0.14726807687149587, 'r': 0.2624053754138888}, 'rouge-2': {'f': 0.05339623221156814, 'p': 0.0428963047088105, 'r': 0.07796638289983866}, 'rouge-l': {'f': 0.16839083703030966, 'p': 0.13567980682616082, 'r': 0.24092795234326472}}
